# Minicaml: Coppie e Destructuring assignment

In questa esercitazione, vediamo insieme alcuni dettagli dell'interprete di Minicaml, e lo estendiamo con la possibilità di **creare coppie di valori** e di **dichiarare due variabili contemporaneamente**.

Un esempio di sintassi concreta di questi due nuovi costrutti può essere:

    (4, 2)                       (*coppia di interi*)
    (4, "ciao")                  (*coppia di un intero e una stringa*)
    let x = (4, 2) in ...        (*assegnamo a x una coppia*)
    let foo n = (n, n) in ...    (*foo è una funzione che resitutisce coppie*)   
    
    let a, b = (4, 2) in a + b   (*dichiariamo due variabili a e b, assegnandovi il primo e secondo valore della coppia*)
    let a, b = foo 5 in a + b    (*dichiariamo due variabili a e b, assegnandovi il risultato della chiamata foo 5*)
    let a, b = 10 in a + b       (*Errore: 10 non è una coppia, il destructuring assignment da' errore*)

## Esercizio 1: Sintassi

Questa è la definizione della sintassi astratta dell'interprete di Minicaml. Il tipo `exp` è un albero: 
 - I nodi costruiti con `EInt` sono foglie, perchè non hanno sottoespressioni, hanno solo una "etichetta" intera
 - I nodi costruiti con `Sum` sono nodi interni, perchè hanno due sottoespressioni
 - I nodi costruiti con `IfThenElse` sono nodi interni, perchè hanno 3 sottoespressioni: guardia, ramo then e ramo else
 - I nodi costruiti con `Let` sono nodi interni, perchè hanno 3 sottoespressioni: variabile da definire, valore della variabile,  continuazione
 - E così via..

Esempio di come si può esprimere la sintassi concreta in singassi astratta:

    let x = 6*7 in x+2     ->     Let("x", Prod(EInt(6), EInt(7)), Sum(Den("x"), EInt(2)))

Estendere questa definizione con due nuovi costrutti, uno per costruire le coppie, uno per il destructuring assignment. 

**HINT**: Per il destructuring assignment, prendere ispirazione dal `let` che già esiste! Cosa va aggiunto? 

In [ ]:
(* Identificatori *)
type ide = string;;


(* Abstract Expressions = espressioni nella sintassi astratta, 
   compongono l'Albero di Sintassi Astratta *)
type exp = 
    | EInt of int
    | CstTrue 
    | CstFalse
    | EString of string
    | Den of ide
    (* Operatori binari da interi a interi*)
    | Sum of exp * exp
    | Diff of exp * exp
    | Prod of exp * exp
    | Div of exp * exp
    (* Operatori da interi a booleani *)
    | IsZero of exp
    | Eq of exp * exp
    | LessThan of exp*exp
    | GreaterThan of exp*exp
    (* Operatori su booleani *)
    | And of exp*exp
    | Or of exp*exp
    | Not of exp
    (* Controllo del flusso, funzioni *)
    | IfThenElse of exp * exp * exp
    | Let of ide * exp * exp
    | Letrec of ide * ide  * exp * exp
    | Fun of ide * exp
    | Apply of exp * exp

## Esercizio 2: valori 

Questa è la definizione di ambiente e di valori esprimibili (`evT`):

* L'ambiente associa identificatori a valori. L'ambiente è implementato come una funzione vera e propria "aggiornabile" tramite la funzione `bind`
* I valori che Minicaml può manipolare sono interi, booleani, stringhe e closure. Ognuno di questi valori di Minicaml viene implementato con un corrispondente tipo di OCaml

Estendere la definizione di `evT` con tutti i nuovi tipi necessari per manipolare le coppie. Quanti sono?

In [ ]:
(* Ambiente polimorfo *)
type 't env = ide -> 't

(* Evaluation types = tipi  esprimibili *)
type evT = 
    | Int of int 
    | Bool of bool 
    | String of string 
    | Closure of ide * exp * evT env 
    | RecClosure of ide * ide * exp * evT env
    | UnBound
    
(* Ambiente vuoto *)
let emptyenv = function x -> UnBound

(* Binding fra una stringa x e un valore primitivo evT *)
let bind (s: evT env) (x: ide) (v: evT) = 
    function (i: ide) -> if (i = x) then v else (s i)

## Esercizio 2.5: Type Checking (facoltativo)

Estendere la definizione di tipi di minicaml `tname` con un tipo per le coppie, ed estendere le altre funzioni di conseguenza.

In [ ]:
(* I tipi *)
type tname =  TInt | TBool | TString | TClosure | TRecClosure | TUnBound

(* Funzione da evT a tname che associa a ogni valore il suo descrittore di tipo  *)
let getType (x: evT) : tname =
    match x with
    | Int(n) -> TInt
    | Bool(b) -> TBool
    | String(s) -> TString
    | Closure(i,e,en) -> TClosure
    | RecClosure(i,j,e,en) -> TRecClosure
    | UnBound -> TUnBound
    
(* Type-checking *)
let typecheck ((x, y) : (tname*evT)) = 
    match x with
    | TInt -> (match y with 
               | Int(u) -> true
               | _ -> false
               )
    | TBool -> (match y with 
                | Bool(u) -> true
                | _ -> false
                )
    | TString -> (match y with
                 | String(u) -> true
                 | _ -> false
                 )
    | TClosure -> (match y with
                   | Closure(i,e,n) -> true
                   | _ -> false
                   )
    | TRecClosure -> (match y with
                     | RecClosure(i,j,e,n) -> true
                     | _ -> false
                     )
    |TUnBound -> (match y with
                 | UnBound -> true
                 | _ -> false
                 )

## Eccezione in caso di errori durante l'esecuzione

In [ ]:
(* Errori a runtime *)
exception RuntimeError of string

## Operazioni primitive

In [ ]:
(* PRIMITIVE del linguaggio *)

(* Controlla se un numero è zero *)
let is_zero(x) = match (typecheck(TInt,x),x) with
    | (true, Int(v)) -> Bool(v = 0)
    | (_, _) -> raise ( RuntimeError "Wrong type")

(* Uguaglianza fra interi *)
let int_eq(x,y) =   
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v = w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Somma fra interi *)	   
let int_plus(x, y) = 
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v + w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Differenza fra interi *)
let int_sub(x, y) = 
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v - w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Prodotto fra interi *)
let int_times(x, y) =
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v * w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")
    
(* Divisione fra interi *)
let int_div(x, y) =
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> 
                    if w<>0 then Int(v / w)
                            else raise (RuntimeError "Division by zero")
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Operazioni di confronto *)
let less_than(x, y) = 
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v < w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let greater_than(x, y) = 
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v > w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Operazioni logiche *)
let bool_and(x,y) = 
    match (typecheck(TBool,x), typecheck(TBool,y), x, y) with
    | (true, true, Bool(v), Bool(w)) -> Bool(v && w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let bool_or(x,y) = 
    match (typecheck(TBool,x), typecheck(TBool,y), x, y) with
    | (true, true, Bool(v), Bool(w)) -> Bool(v || w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let bool_not(x) = 
    match (typecheck(TBool,x), x) with
    | (true, Bool(v)) -> Bool(not(v))
    | (_,_) -> raise ( RuntimeError "Wrong type")

## Esercizio 3: Interprete

Questa è la definizione del cuore dell'interprete, ovvero la funzione `eval e s`, che prende un espressione `e` e la valuta in un ambiente `s`, restituendo un valore di tipo `evT`.

La funzione `eval` è una visita post order dell'albero: per restituire il valore di un nodo interno, calcola ricorsivamente il valore delle sottoespressioni, e poi li "combina" in qualche modo per restituire il valore del noto interno.
 - `eval Sum(e1, e2) s` calcola il valore di `e1`, il valore di `e2`, e li somma
 - `eval Apply(eF, eArg) s` calcola il valore di `ef` (che sarà una closure), il valore di `e2`, e applica la closure al valore di `e2`
 - `eval Let("x", e, body) s` calcola il valore di `e`, aggiorna l'ambiente `s` con un binding fra `x` e il valore di `e`, e in questo nuovo ambiente esegue il body. 
 
Estendere questa funzione con le espressioni definite nell'esercizio 1, facendo uso dei valori definiti nell'esercizio 2. Come prima, per il destructuring assignment prendere ispirazione dalla semantica di `let`

In [ ]:
(* Interprete *)
let rec eval (e:exp) (s:evT env) : evT = 
    match e with
    | EInt(n) -> Int(n)
    | CstTrue -> Bool(true)
    | CstFalse -> Bool(false)
    | EString(s) -> String(s)
    | Den(i) -> (s i)

    | Prod(e1,e2) -> int_times((eval e1 s), (eval e2 s))
    | Sum(e1, e2) -> int_plus((eval e1 s), (eval e2 s))
    | Diff(e1, e2) -> int_sub((eval e1 s), (eval e2 s))
    | Div(e1, e2) -> int_div((eval e1 s), (eval e2 s))

    | IsZero(e1) -> is_zero (eval e1 s)
    | Eq(e1, e2) -> int_eq((eval e1 s), (eval e2 s))
    | LessThan(e1, e2) -> less_than((eval e1 s),(eval e2 s))
    | GreaterThan(e1, e2) -> greater_than((eval e1 s),(eval e2 s))

    | And(e1, e2) -> bool_and((eval e1 s),(eval e2 s))
    | Or(e1, e2) -> bool_or((eval e1 s),(eval e2 s))
    | Not(e1) -> bool_not(eval e1 s)

    | IfThenElse(e1,e2,e3) -> 
        let g = eval e1 s in 
            (match (typecheck(TBool,g),g) with
            |(true, Bool(true)) -> eval e2 s
            |(true, Bool(false)) -> eval e3 s
            |(_,_) -> raise ( RuntimeError "Wrong type")
            )
    | Let(i, e, ebody) -> eval ebody (bind s i (eval e s))
    | Fun(arg, ebody) -> Closure(arg,ebody,s)
    | Letrec(f, arg, fBody, leBody) ->
        let benv = bind (s) (f) (RecClosure(f, arg, fBody,s)) in
            eval leBody benv
    | Apply(eF, eArg) ->
        let fclosure = eval eF s in 
            (match fclosure with 
            | Closure(arg, fbody, fDecEnv) -> 
                let aVal = eval eArg s in 
                let aenv = bind fDecEnv arg aVal in 
                eval fbody aenv 
            | RecClosure(f, arg, fbody, fDecEnv) -> 
                let aVal = eval eArg s in
                let rEnv = bind fDecEnv f fclosure in
                let aenv = bind rEnv arg aVal in 
                eval fbody aenv
            | _ -> raise ( RuntimeError "Wrong type")
            );;